In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
sample_sub = pd.read_csv("sample_submission.csv")
train_df = pd.read_csv("train (1).csv")
test_df = pd.read_csv("test (1).csv")

In [4]:
X_train = train_df.drop(columns={'Id','Property_A','Property_B','Property_C','Property_D','Property_E','Property_F'})
y_train = train_df[['Property_A','Property_B','Property_C','Property_D','Property_E','Property_F']]

In [5]:
sensor1_X_train = X_train[['A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']]

In [6]:
sensor2_X_train = X_train.drop(columns={'A ','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'})

In [101]:
copy_sensor2_X_train = sensor2_X_train


# https://lifewithdata.com/2022/03/13/how-to-remove-highly-correlated-features-from-a-dataset/
corr_matrix = sensor2_X_train.corr().abs()

# select upper traingle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

<ipython-input-101-63e72ba12461>:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [102]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.9999998)]

In [104]:
# drop the columns
copy_sensor2_X_train.drop(to_drop, axis=1, inplace=True)

In [105]:
result = pd.concat([sensor1_X_train, copy_sensor2_X_train], axis=1)

In [87]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train_1)

# transform training data
X_train_norm = norm.transform(X_train_1)
X_train_norm_df = pd.DataFrame(X_train_norm)

# transform testing dataabs
X_test_norm = norm.transform(X_test)
X_test_norm_df = pd.DataFrame(X_test_norm)

In [88]:
#  Standardize
from sklearn.preprocessing import StandardScaler
 
scalar = StandardScaler()
 
# fitting
scalar.fit(X_train_norm_df)
X_train_stan = scalar.transform(X_train_norm_df)
X_train_stan_df = pd.DataFrame(X_train_stan)

scalar.fit(X_test_norm_df)
X_test_stan = scalar.transform(X_test_norm_df)
X_test_stan_df = pd.DataFrame(X_test_stan)

In [108]:
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgm
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

clf_1 = MultiOutputRegressor(lgm.LGBMRegressor())
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=10, random_state=0))
# clf = MultiOutputRegressor(xgb.XGBRegressor(eval_metric='rmse'))
# clf = MultiOutputRegressor(xgb.XGBRegressor())
clf.fit(X_train_1, y_train_1)
clf_1.fit(X_train_stan_df, y_train_1)

MultiOutputRegressor(estimator=LGBMRegressor())

In [109]:
y_preds_1 = clf_1.predict(X_test_stan_df)
y_preds = clf.predict(X_test)

In [110]:
final_preds = (y_preds + y_preds_1)/2

In [111]:
mean_squared_error(y_test, final_preds)

28.83580669260924